# Merging all InfoUSA files and saving them as a ```.parquet``` file

### Import statements

In [1]:
import pyarrow
import re
import os
import pandas as pd

### Merging 00000-25000
We initially attempted to merge all files in one go and save the combined file as a ```.csv()``` file. Although we were able to do this, reading in the combined ```.csv()``` file would not work- our kernel crashed every time we attempted it, even using 38 CPU cores and 208GB of RAM. 

Therefore, we decided to use pyrarrow as an engine when reading in each of the ```.txt``` files and save the combined file as a ```.parquet()``` file. Pyarrow allows for multithreading, leading to a more efficient use of CPU cores, and Parquet files are optimized for storing large datasets. This is because Parquet files are column-based while CSV files are row-based, meaning it can only focus on the data relevant to the user's query. Like this, it only loads into memory the columns of the data that are being used. Parquet files can also be read in and exported using pandas, which is an added bonus when working with pandas dataframes.

Still, we ran into memory issues merging all files in one go- so we broke up our data into four sections, merged the files in each of these sections, dropped columns not relevant to our work, then combined the four sections for one totally combined dataframe.

Here, we are merging zip codes in the range 00000 - 25000. The regular expression sifts through the list of all files in the InfoUSA folder and creates a list of only the files with zip codes in that range.

In [3]:
os.chdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
directory = os.listdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
r = re.compile("Household_Ethnicity_zip_(0{4}[0-9]|0{3}[1-9][0-9]|00[1-9][0-9]{2}|0[1-9][0-9]{3}|1[0-9]{4}|2[0-4][0-9]{3}|25000)_year_2020.txt")
newlist = list(filter(r.match, directory)) # Read Note
# print(newlist)
len(newlist)

38248


8706

Then, we use the pandas ```.concat()``` function to concatenate every file in the list of files created above.

In [3]:
%%time
#combine all files in the list
zip_00_25 = pd.concat([pd.read_csv(f, sep = '\t', engine='pyarrow') for f in newlist])
zip_00_25.head()

CPU times: user 6min 19s, sys: 2min 37s, total: 8min 57s
Wall time: 9min 33s


,FamilyID,location_type,primary_family_ind,HouseholdStatus,tradeline_count,head_hh_age_code,length_of_residence,ChildrenHHCount,children_ind,AddressType,...,GE_CENSUS_LEVEL_2010,middle_name,last_name_1,Ethnicity_Code_1,first_name_2,last_name_2,Ethnicity_Code_2,first_name_3,last_name_3,Ethnicity_Code_3
0,101133133,S,1,F,0,K,2,0,0,S,...,P,,TODD,E5,DEBORAH,TODD,E5,KRISTIN,TODD,E5
1,105579259,U,0,F,0,H,9,0,0,P,...,Z,M,WOOD,E5,WENDY,WOOD,E5,,,
2,282131063,S,1,F,0,C,3,1,1,S,...,P,S,BRADY,IE,THOMAS,BRADY,IE,NITA,BRADY,IE
3,340638960,S,1,F,0,L,27,0,0,S,...,P,S,MAGEE,IE,RUTH,MAGEE,IE,,,
4,398667313,S,1,F,0,H,33,1,1,S,...,P,J,FINAN,IE,,,,,,


Filter for only the columns relevant to our future work:

In [4]:
%%time
zip_00_25 = zip_00_25[['ZIP', 'census_county_2010', 'STATE', 'ChildrenHHCount', 'children_ind', 'head_hh_age_code', 'GE_LATITUDE_2010', 'GE_LONGITUDE_2010']]

CPU times: user 16.6 s, sys: 36.2 s, total: 52.8 s
Wall time: 53.3 s


Filter for only zipcodes not in Alaska, Hawaii, Puerto Rico, and the Virgin Islands, as there are no storage tanks in the AST dataset from these areas:

In [5]:
%%time
zip_00_25 = zip_00_25[((zip_00_25['STATE'] != 'AK') & (zip_00_25['STATE'] != 'HI') & (zip_00_25['STATE'] != 'PR') & (zip_00_25['STATE'] != 'VI'))]

,ZIP,census_county_2010,STATE,ChildrenHHCount,children_ind,head_hh_age_code,GE_LATITUDE_2010,GE_LONGITUDE_2010
0,18833,113,PA,0,0,K,41.546738,-76.540436
1,18833,15,PA,0,0,H,41.590800,-76.424200
2,18833,15,PA,1,1,C,41.600392,-76.441724
3,18833,15,PA,0,0,L,41.592483,-76.437832
4,18833,15,PA,1,1,H,41.566196,-76.347977
...,...,...,...,...,...,...,...,...
1528,18322,89,PA,0,0,G,40.927534,-75.401987
1529,18322,89,PA,0,0,E,40.926938,-75.375047
1530,18322,89,PA,0,0,E,40.912001,-75.377969
1531,18322,89,PA,0,0,B,40.918671,-75.388654


Write out the dataframe as a ```.parquet()``` file:

In [6]:
%%time
zip_00_25.to_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_00_25_final.parquet')

CPU times: user 10.4 s, sys: 638 ms, total: 11 s
Wall time: 12.1 s


### Merging 25000-50000
We use the same steps as Merging 00000 - 25000 section.

In [2]:
os.chdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
directory = os.listdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
r = re.compile("Household_Ethnicity_zip_(2500[0-9]|250[1-9][0-9]|25[1-9][0-9]{2}|2[6-9][0-9]{3}|[34][0-9]{4}|50000)_year_2020.txt")
newlist = list(filter(r.match, directory)) # Read Note
# print(newlist)
len(newlist)

10758

In [3]:
%%time
#combine all files in the list
zip_25_50 = pd.concat([pd.read_csv(f, sep = '\t', engine='pyarrow') for f in newlist])
zip_25_50.head()

CPU times: user 8min 41s, sys: 3min 19s, total: 12min 1s
Wall time: 12min 6s


,FamilyID,location_type,primary_family_ind,HouseholdStatus,tradeline_count,head_hh_age_code,length_of_residence,ChildrenHHCount,children_ind,AddressType,...,GE_CENSUS_LEVEL_2010,middle_name,last_name_1,Ethnicity_Code_1,first_name_2,last_name_2,Ethnicity_Code_2,first_name_3,last_name_3,Ethnicity_Code_3
0,204505,S,1,F,2,G,19,0,0,S,...,P,R,DEFEE,E5,JACQUELINE,DEFEE,E5,,,
1,207105250,S,1,F,0,H,11,1,1,S,...,P,M,SLACK,B5,YVONNE,SLACK,S3,ANDREA,SLACK,S3
2,210892206,S,1,F,0,F,11,5,1,S,...,P,I,ROWE,E5,REBEKAH,ROWE,E5,MEGAN,ROWE,E5
3,210892421,S,1,F,0,H,21,1,1,S,...,P,R,ROEHLER,DE,ANITA,ROEHLER,DE,MIRANDA,ROEHLER,DE
4,210920662,U,1,F,0,I,26,0,0,P,...,Z,K,ROBY,E5,MELISSA,ROBY,E5,,,


In [4]:
%%time
zip_25_50 = zip_25_50[['ZIP', 'census_county_2010', 'STATE', 'ChildrenHHCount', 'children_ind', 'head_hh_age_code', 'GE_LATITUDE_2010', 'GE_LONGITUDE_2010']]

CPU times: user 22.1 s, sys: 1min 3s, total: 1min 25s
Wall time: 1min 25s


In [5]:
%%time
zip_25_50 = zip_25_50[((zip_25_50['STATE'] != 'AK') & (zip_25_50['STATE'] != 'HI') & (zip_25_50['STATE'] != 'PR') & (zip_25_50['STATE'] != 'VI'))]

CPU times: user 13.8 s, sys: 744 ms, total: 14.5 s
Wall time: 14.5 s


In [7]:
%%time
zip_25_50.to_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_25_50_final.parquet')

CPU times: user 14.2 s, sys: 1.06 s, total: 15.2 s
Wall time: 16.2 s


### Merging 50000-75000
We use the same steps as Merging 00000 - 25000 section.

In [2]:
os.chdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
directory = os.listdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
r = re.compile("Household_Ethnicity_zip_(5000[0-9]|500[1-9][0-9]|50[1-9][0-9]{2}|5[1-9][0-9]{3}|6[0-9]{4}|7[0-4][0-9]{3}|75000)_year_2020.txt")
newlist = list(filter(r.match, directory)) # Read Note
# print(newlist)
len(newlist)

9917

In [3]:
%%time
#combine all files in the list
zip_50_75 = pd.concat([pd.read_csv(f, sep = '\t', engine='pyarrow') for f in newlist])
zip_50_75.head()

CPU times: user 5min 15s, sys: 2min 14s, total: 7min 30s
Wall time: 9min 6s


,FamilyID,location_type,primary_family_ind,HouseholdStatus,tradeline_count,head_hh_age_code,length_of_residence,ChildrenHHCount,children_ind,AddressType,...,GE_CENSUS_LEVEL_2010,middle_name,last_name_1,Ethnicity_Code_1,first_name_2,last_name_2,Ethnicity_Code_2,first_name_3,last_name_3,Ethnicity_Code_3
0,52749969,S,1,F,0,L,11,0,0,S,...,P,A,KNIGHT,W4,SHARON,KNIGHT,W4,SHERRY,KNIGHT,W4
1,175470537,S,0,F,0,L,19,1,1,S,...,P,R,HAZLETON,E5,,,,,,
2,175525274,S,1,F,2,M,18,0,0,S,...,P,,ASHFORD,E5,WALTER,ASHFORD,E5,JULIE,ASHFORD,E5
3,175566359,S,1,F,0,M,29,0,1,S,...,P,A,BARTLETT,E5,LINDA,BARTLETT,E5,,,
4,175566362,S,1,F,3,H,17,2,1,S,...,P,,BOGGS,E5,MARIE,BOGGS,E5,VIOLET,BOGGS,E5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,902430603549,U,1,F,0,C,1,0,0,P,...,Z,,BOSLEY,E5,,,,,,
652,902430603557,S,1,F,0,A,1,0,0,S,...,0,,MARFILIUS,00,,,,,,
653,902430603565,U,0,F,0,J,1,0,0,P,...,Z,,WARRINGTON,E5,,,,,,
654,902454354841,U,1,F,0,G,1,0,0,,...,Z,,QUICK,E5,,,,,,


In [4]:
%%time
zip_50_75 = zip_50_75[['ZIP', 'census_county_2010', 'STATE', 'ChildrenHHCount', 'children_ind', 'head_hh_age_code', 'GE_LATITUDE_2010', 'GE_LONGITUDE_2010']]

CPU times: user 11.9 s, sys: 12.3 s, total: 24.2 s
Wall time: 24.2 s


In [5]:
%%time
zip_50_75 = zip_50_75[((zip_50_75['STATE'] != 'AK') & (zip_50_75['STATE'] != 'HI') & (zip_50_75['STATE'] != 'PR') & (zip_50_75['STATE'] != 'VI'))]
zip_50_75

CPU times: user 6.98 s, sys: 304 ms, total: 7.29 s
Wall time: 7.27 s


,ZIP,census_county_2010,STATE,ChildrenHHCount,children_ind,head_hh_age_code,GE_LATITUDE_2010,GE_LONGITUDE_2010
0,74869,81,OK,0,0,L,35.607354,-96.824352
1,74869,81,OK,1,1,L,35.548371,-96.727254
2,74869,81,OK,0,0,M,35.571250,-96.696193
3,74869,81,OK,0,1,M,35.568520,-96.757917
4,74869,81,OK,2,1,H,35.607481,-96.761271
...,...,...,...,...,...,...,...,...
651,53801,43,WI,0,0,C,42.920500,-91.072000
652,53801,43,WI,0,0,A,42.904946,-91.104199
653,53801,43,WI,0,0,J,42.920500,-91.072000
654,53801,43,WI,0,0,G,42.920500,-91.072000


In [6]:
%%time
zip_50_75.to_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_50_75_final.parquet')

CPU times: user 7.3 s, sys: 397 ms, total: 7.69 s
Wall time: 7.97 s


### Merging 75000-99000
We use the same steps as Merging 00000 - 25000 section.

In [2]:
os.chdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
directory = os.listdir("/hpc/group/codeplus22-vis/infousa_copy/2020")
r = re.compile("Household_Ethnicity_zip_(7500[0-9]|750[1-9][0-9]|75[1-9][0-9]{2}|7[6-9][0-9]{3}|[89][0-9]{4})_year_2020.txt")
newlist = list(filter(r.match, directory)) # Read Note
# print(newlist)
len(newlist)

8824

In [4]:
%%time
#combine all files in the list
zip_75_99 = pd.concat([pd.read_csv(f, sep = '\t', engine='pyarrow') for f in newlist])
zip_75_99.head()

CPU times: user 8min 9s, sys: 2min 53s, total: 11min 2s
Wall time: 10min 34s


,FamilyID,location_type,primary_family_ind,HouseholdStatus,tradeline_count,head_hh_age_code,length_of_residence,ChildrenHHCount,children_ind,AddressType,...,GE_CENSUS_LEVEL_2010,middle_name,last_name_1,Ethnicity_Code_1,first_name_2,last_name_2,Ethnicity_Code_2,first_name_3,last_name_3,Ethnicity_Code_3
0,25899683,U,1,F,0,M,30,0,0,P,...,Z,J,LEMN,00,,,,,,
1,28681487,U,1,F,0,I,30,0,0,P,...,Z,J,MASTERS,E5,,,,,,
2,28681665,U,1,F,0,I,24,1,1,P,...,Z,K,CHENG,CN,LILY,CHENG,CN,,,
3,28681725,U,1,F,0,K,25,0,0,P,...,Z,W,DAWS,E5,,,,,,
4,28681762,U,1,F,0,H,22,0,0,P,...,Z,P,RAMOS,H5,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174,902449020153,S,0,F,0,C,1,0,0,S,...,P,,AVILA,H5,,,,,,
3175,902449020161,S,0,F,0,E,1,0,0,S,...,Z,,CHAN,CN,,,,,,
3176,902449020179,S,1,F,0,G,1,0,0,S,...,P,,FAIN,FR,,,,,,
3177,902449020187,M,1,F,0,H,1,0,0,S,...,P,,FOSTER,S3,,,,,,


In [5]:
%%time
zip_75_99 = zip_75_99[['ZIP', 'census_county_2010', 'STATE', 'ChildrenHHCount', 'children_ind', 'head_hh_age_code', 'GE_LATITUDE_2010', 'GE_LONGITUDE_2010']]

CPU times: user 21.7 s, sys: 22.5 s, total: 44.2 s
Wall time: 44.3 s


In [6]:
%%time
zip_75_99 = zip_75_99[((zip_75_99['STATE'] != 'AK') & (zip_75_99['STATE'] != 'HI') & (zip_75_99['STATE'] != 'PR') & (zip_75_99['STATE'] != 'VI'))]

CPU times: user 13.1 s, sys: 499 ms, total: 13.6 s
Wall time: 13.5 s


In [8]:
%%time
zip_75_99.to_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_75_99_final.parquet')

CPU times: user 13.2 s, sys: 652 ms, total: 13.9 s
Wall time: 15.4 s


### Merging 00000 - 50000
Now that we merged each of our four sections, we will merge these four sections with eachother using pandas' ```.append()``` function.

In [4]:
%%time
zip_00_50 = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_00_25_final.parquet').append(pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_25_50_final.parquet'), ignore_index = True)
zip_00_50

<timed exec>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


CPU times: user 17 s, sys: 5.57 s, total: 22.6 s
Wall time: 20 s


,ZIP,census_county_2010,STATE,ChildrenHHCount,children_ind,head_hh_age_code,GE_LATITUDE_2010,GE_LONGITUDE_2010
0,18833,113,PA,0,0,K,41.546738,-76.540436
1,18833,15,PA,0,0,H,41.590800,-76.424200
2,18833,15,PA,1,1,C,41.600392,-76.441724
3,18833,15,PA,0,0,L,41.592483,-76.437832
4,18833,15,PA,1,1,H,41.566196,-76.347977
...,...,...,...,...,...,...,...,...
103993979,39474,65,MS,0,0,F,31.613300,-89.856400
103993980,39474,65,MS,0,0,B,31.596286,-89.866823
103993981,39474,65,MS,0,0,F,31.662703,-89.789536
103993982,39474,65,MS,0,0,F,31.610200,-89.871200


In [5]:
%%time
zip_00_50.to_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_00_50_final.parquet')

CPU times: user 22.1 s, sys: 1.17 s, total: 23.3 s
Wall time: 24.1 s


### Merging 50000 - 99000
We use the same steps as Merging 00000 - 50000 section.

In [4]:
%%time
zip_50_99 = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_50_75_final.parquet').append(pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_75_99_final.parquet'), ignore_index = True)
zip_50_99

<timed exec>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


CPU times: user 14.4 s, sys: 4.69 s, total: 19.1 s
Wall time: 10.4 s


,ZIP,census_county_2010,STATE,ChildrenHHCount,children_ind,head_hh_age_code,GE_LATITUDE_2010,GE_LONGITUDE_2010
0,74869,81,OK,0,0,L,35.607354,-96.824352
1,74869,81,OK,1,1,L,35.548371,-96.727254
2,74869,81,OK,0,0,M,35.571250,-96.696193
3,74869,81,OK,0,1,M,35.568520,-96.757917
4,74869,81,OK,2,1,H,35.607481,-96.761271
...,...,...,...,...,...,...,...,...
86993624,92003,73,CA,0,0,C,33.285885,-117.240445
86993625,92003,73,CA,0,0,E,33.284700,-117.210800
86993626,92003,73,CA,0,0,G,33.282869,-117.183963
86993627,92003,73,CA,0,0,H,33.278284,-117.181181


In [5]:
%%time
zip_50_99.to_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_50_99_final.parquet')

CPU times: user 18.6 s, sys: 912 ms, total: 19.5 s
Wall time: 21 s


### Merging 00-99
We use the same steps as Merging 00000 - 25000 section, but this time merge 00000 - 50000 and 50000 - 99999 to get our final, totally combined dataframe.

In [3]:
%%time
zip_00_99 = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_00_50_final.parquet').append(pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_50_99_final.parquet'), ignore_index = True)
zip_00_99

<timed exec>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


CPU times: user 30.5 s, sys: 9.55 s, total: 40 s
Wall time: 21.4 s


,ZIP,census_county_2010,STATE,ChildrenHHCount,children_ind,head_hh_age_code,GE_LATITUDE_2010,GE_LONGITUDE_2010
0,18833,113,PA,0,0,K,41.546738,-76.540436
1,18833,15,PA,0,0,H,41.590800,-76.424200
2,18833,15,PA,1,1,C,41.600392,-76.441724
3,18833,15,PA,0,0,L,41.592483,-76.437832
4,18833,15,PA,1,1,H,41.566196,-76.347977
...,...,...,...,...,...,...,...,...
190987608,92003,73,CA,0,0,C,33.285885,-117.240445
190987609,92003,73,CA,0,0,E,33.284700,-117.210800
190987610,92003,73,CA,0,0,G,33.282869,-117.183963
190987611,92003,73,CA,0,0,H,33.278284,-117.181181


### Renaming columns
We rename the columns in our dataset for standardization purposes.

In [6]:
zip_00_99.rename(columns = {'ZIP': 'zip', 'census_county_2010': 'county', 'STATE': 'state', 'ChildrenHHCount': 'child_num', 
                           'children_ind': 'has_child', 'head_hh_age_code': 'age_code', 'GE_LATITUDE_2010': 'lat_h_4326', 
                            'GE_LONGITUDE_2010': 'lon_h_4326'}, inplace = True)
zip_00_99

,zip,county,state,child_num,has_child,age_code,lat_h_4326,lon_h_4326
0,18833,113,PA,0,0,K,41.546738,-76.540436
1,18833,15,PA,0,0,H,41.590800,-76.424200
2,18833,15,PA,1,1,C,41.600392,-76.441724
3,18833,15,PA,0,0,L,41.592483,-76.437832
4,18833,15,PA,1,1,H,41.566196,-76.347977
...,...,...,...,...,...,...,...,...
190987608,92003,73,CA,0,0,C,33.285885,-117.240445
190987609,92003,73,CA,0,0,E,33.284700,-117.210800
190987610,92003,73,CA,0,0,G,33.282869,-117.183963
190987611,92003,73,CA,0,0,H,33.278284,-117.181181


### Transforming household latitude and longitude coordinates from EPSG 4326 to EPSG 3857
A lot of our visualizations need coordinates in EPSG 3857, however these coordinates are in EPSG 4326. Therefore, we use the pyproj interface, which allows us to use the PROJ coordinate transformation software to transform our EPSG 4326 coordinates to EPSG 3857. This creates two new columns in our original dataset with the transformed coordinates.

In [9]:
from pyproj import Proj, Transformer

In [11]:
# Apply transformation
transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')
zip_00_99['lat_h_3857'], zip_00_99['lon_h_3857'] = transform_4326_to_3857.transform(
                                                zip_00_99['lat_h_4326'], zip_00_99['lon_h_4326'])

zip_00_99

,zip,county,state,child_num,has_child,age_code,lat_h_4326,lon_h_4326,lat_h_3857,lon_h_3857
0,18833,113,PA,0,0,K,41.546738,-76.540436,-8.520442e+06,5.093323e+06
1,18833,15,PA,0,0,H,41.590800,-76.424200,-8.507503e+06,5.099879e+06
2,18833,15,PA,1,1,C,41.600392,-76.441724,-8.509454e+06,5.101307e+06
3,18833,15,PA,0,0,L,41.592483,-76.437832,-8.509021e+06,5.100129e+06
4,18833,15,PA,1,1,H,41.566196,-76.347977,-8.499018e+06,5.096218e+06
...,...,...,...,...,...,...,...,...,...,...
190987608,92003,73,CA,0,0,C,33.285885,-117.240445,-1.305115e+07,3.933312e+06
190987609,92003,73,CA,0,0,E,33.284700,-117.210800,-1.304785e+07,3.933154e+06
190987610,92003,73,CA,0,0,G,33.282869,-117.183963,-1.304486e+07,3.932911e+06
190987611,92003,73,CA,0,0,H,33.278284,-117.181181,-1.304455e+07,3.932300e+06


### Exporing final dataframe
Finally, we export this dataframe as a ```.parquet()``` file for further use.

In [ ]:
%%time
zip_00_99.to_parquet('/hpc/group/codeplus22-vis/infousa_copy/zip_00_99_final.parquet')